# Mathe2 MiniTest 5

## By [@O-Manoli](https://github.com/O-Manoli)

In [ ]:
import math
import sympy as sp
from typing import Callable
import typing
from itertools import chain
from urllib.parse import quote
from functools import partial, wraps
from IPython.display import display
from IPython.core.display import Math, Latex, Markdown

# - - - -

from sympy import pi, Rational as R

# - - - - - - - - -

###############
### Globals ###
###############


# shorthand

def L(Obj, *args, **kwargs) -> str: return sp.latex(Obj, *args, **kwargs)

def P(STR:str, *args, **kwargs):        display(Math(STR, *args, **kwargs))
def PL(STR:str, *args, **kwargs):       display(Latex(STR, *args, **kwargs))
def PM(STR:str, *args, **kwargs):       display(Markdown(STR, *args, **kwargs))

def PF(*args, Format:str | None = None):
	LF = lambda O: L(O) if not isinstance(O, str) else O # Latex-Filter
	Format = " ".join(len(args)*["{}"]) if Format is None else Format
	P(Format.format(*map(LF, args)))

# - - - - - - - - -

## Wolfram-Alpha

def WRALink(CMD:str)->str:
	return r"https://www.wolframalpha.com/input/?i=" + quote(CMD)

def WRAMarkdown(
		CMD:str,
		Format:str = "#### [{}]({})",
		FormatLink:Callable[[str], str] = lambda CMD: f"WRA: {CMD}",
		WRALink:Callable[[str], str] = WRALink
		):
	PM(Format.format(FormatLink(CMD), WRALink(CMD)))

WRAMarkdownMath = partial(WRAMarkdown, FormatLink = lambda CMD: f"WRA: ${CMD}$")

# - - - - - - - - - - - - - - - - - - - - -

#################
### Aufgabe 1 ###
#################

def dontSimplify(f:Callable):
	@wraps(f)
	def wrapped(*args, **kwargs):
		return f(*args, evaluate = False,**kwargs) # disable automatic simplification
	return wrapped
## try:
# 10*cos(t + 3*pi/5)
# 10*sp.cos(t + 3*pi/5)

cos = dontSimplify(sp.cos)
sin = dontSimplify(sp.sin)

def Terms(Type):
	def extractFrom(expr:sp.Expr):
		return [term for term in expr.as_terms()[-1] if isinstance(term, Type)]
	return extractFrom

def Phase(Trig:sp.cos | sp.sin, t:sp.Symbol) -> sp.Expr:
	return 2*pi/Trig.args[0].coeff(t) # type: ignore

CosTerms = Terms(sp.cos)
SinTerms = Terms(sp.sin)

def A1(expr:sp.Expr, t:sp.Symbol):
	PM(f"## kleinste positive Periode $T$ der Funktion $f$")
	P(f"f({L(t)}) = {L(expr)}")
	TrigTerms = tuple(chain(CosTerms(expr), SinTerms(expr)))
	kgv = [Phase(term, t) for term in TrigTerms]
	for i, (term, T) in enumerate(zip(TrigTerms, kgv), 1):
		PM(f"${L(term)}$ hat die Periode $T_{{{i}}} = {T} $")
	ANS = math.prod(kgv)
	P(f"T = kgv({', '.join(L(v) for v in kgv)}) = {ANS}")

	PM(f"## Plot")
	sp.plot(expr, (t, 0, ANS))

	PM(f"## Wolfram Alpha")
	for term in TrigTerms:
		WRAMarkdown(f"period ${L(term)}$")
	WRAMarkdown(f"period ${L(expr)}$")

# - - - - - - - - - - - -

#################
### Aufgabe 2 ###
#################

def hack_intervals(f:sp.Piecewise):
	def fetch_intervals(expr):
		return [bound.args[-1] for bound in expr.args[-1].args[::-1]]
	return tuple(map(fetch_intervals, f.args))

def PIntegral(f:sp.Expr, a, b) -> str:
	return f"\\int_{{{a}}}^{{{b}}} {{{L(f)}}} dt"

def FuncIntervals(f:sp.Piecewise)->list:
	I = hack_intervals(f)
	F = [i[0] for i in f.args]  # type: ignore
	return [(fi, a, b) for fi, (a, b) in zip(F, I)]

def SumForm(Vals:typing.Sequence)->str:
	if not len(Vals):
		return ''
	signed = lambda x: L(x) if x < 0 else f"+ {L(x)}"
	return L(Vals[0]) + str().join(map(signed, Vals[1:]))

def A2(f:sp.Piecewise, t:sp.Symbol):

	PM(f"## Mittelwert $m$ der Funktion $f$")

	Intervals = sorted(set(chain(*hack_intervals(f))))

	T = Intervals[-1] - Intervals[0] # type: ignore

	T_ = R(1, T)

	P(f"f({L(t)}) = {L(f)}")

	P(f"f(t + T) = f(t + {L(T)}) = f(t)")

	FIntervals = FuncIntervals(f)

	Vals = list()

	for i, (fi, a, b) in enumerate(FIntervals, 1):
		Vals.append(sp.integrate(f, (t, a, b)))
		P(f"I_{{{i}}} = {PIntegral(fi, a, b)} = {L(Vals[-1])}")

	ANS = T_*sp.integrate(f, (t, Intervals[0], Intervals[-1]))

	P(f"m = {L(T_)} \\cdot ({SumForm(Vals)}) = {L(ANS)}")

	PM(f"## Plot")

	sp.plot(f, (t, Intervals[0], Intervals[-1]))


# - - - - - - - - - - - -

#################
### Aufgabe 3 ###
#################

def fourier_coef(f,t:sp.Symbol, k, T):
	w = 2*pi/T
	I = lambda fi: sp.integrate(R(2, T)*f*fi, (t, -R(T, 2), R(T, 2)))
	def b(k):
		return I(sp.sin(k*t*w))
	def a(k):
		return I(sp.cos(k*t*w))
	return a(k), b(k)

def A3(f:sp.Piecewise, t:sp.Symbol, *ab):

	PM(f"## Fourier-Koeffizienten der Funktion $f$")

	Intervals = sorted(set(chain(*hack_intervals(f))))

	T = Intervals[-1] - Intervals[0] # type: ignore

	P(f"f({L(t)}) = {L(f)}")

	P(f"f(t + T) = f(t + {L(T)}) = f(t)")

	PM(f"## Koeffizienten")

	C = lambda k: fourier_coef(f, t, k, T)

	for i, (a, b) in zip(ab, map(C, ab)):
		P(f"a_{{{i}}} = {L(a)}" + r"\hspace{2cm}" + f"b_{{{i}}} = {L(b)}")

	PM(f"## Plot")

	sp.plot(f, (t, Intervals[0], Intervals[-1]))


# - - - - - - - - - - - -

#################
### Aufgabe 4 ###
#################

def fourier_coef_all(f,t:sp.Symbol, k, T):
	w = 2*pi/T
	I = lambda fi: sp.integrate(R(2, T)*f*fi, (t, -R(T, 2), R(T, 2)))
	def b(k):
		return I(sp.sin(k*t*w))
	def a(k):
		return I(sp.cos(k*t*w))
	def c(k):
		return sp.integrate(R(1, T)*f*sp.exp(-sp.I*k*w*t), (t, -R(T, 2), R(T, 2))).simplify()
	return a(k), b(k), c(k)


def A4(f:sp.Piecewise, t:sp.Symbol, *ab):

	PM(f"## komplexen Fourier-Koeffizienten der Funktion $f$")

	Intervals = sorted(set(chain(*hack_intervals(f))))

	T = Intervals[-1] - Intervals[0] # type: ignore

	P(f"f({L(t)}) = {L(f)}")

	P(f"f(t + T) = f(t + {L(T)}) = f(t)")

	PM(f"## Koeffizienten")

	C = lambda k: fourier_coef_all(f, t, k, T)

	for i, (a, b, c) in zip(ab, map(C, ab)):
		P(
			r"\hspace{2cm}".join([
				f"a_{{{i}}} = {L(a)}",
				f"b_{{{i}}} = {L(b)}",
				f"c_{{{i}}} = {L(c)}"
			])
		)

	PM(f"## Plot")

	sp.plot(f, (t, Intervals[0], Intervals[-1]))


# - - - - - - - - -

#################
### Aufgabe 5 ###
#################

def A5(f:sp.Piecewise, t:sp.Symbol):

	PM(f"## Das Gibbssche Phänomen $f$")

	Intervals = sorted(set(chain(*hack_intervals(f))))

	a, b = Intervals[0], Intervals[-1]

	T = b - a # type: ignore

	P(f"f({L(t)}) = {L(f)}")

	P(f"f(t + T) = f(t + {L(T)}) = f(t)")

	PM(f"## Überschwinger Näherung")

	f_ = f.args[0][0] # type: ignore

	P(f"t_k = {L(Intervals[-1])} \\cdot k")
	P(
		"\\Delta f = " + f"0.18 | f({L(a)}^+) - f({L(b)}^-)| = " +
		f"0.18 \\cdot |({L(la:=f_.subs(t, a))}) - ({L(lb := f_.subs(t, b))})| = " +
		f"{L(abs(0.18*(la - lb)))}" # type: ignore
	)


# - - - - - - - - -

#################
### Aufgabe 6 ###
#################

def A6(a_k:sp.Sum, b_k:sp.Sum, a02:sp.Number, k:sp.Symbol, t:sp.Symbol, *V:int):
	def A_k():
		def fac(S:sp.Sum):
			f = lambda term: 1 if isinstance(term, sp.cos) or isinstance(term, sp.sin) else term
			return math.prod([f(term) for term in S.args[0].args])
		a = fac(a_k.as_terms()[-1][0])
		b = fac(b_k.as_terms()[-1][0])


		A = sp.sqrt(a**2 + b**2) # type: ignore
		P(
			f"A_k = \\sqrt{{a_k^2 + b_k^2}} = " +
			f" \\sqrt{{({L(a)})^{{2}} + ({L(b)})^{{2}}}} = " +
			f" \\sqrt{{{L(a**2)} + {L(b**2)}}} = {L(A)}" # type: ignore
		)
		def Ak(i:int):
			if i != 0:
				return A.limit(k, i)
			return 2*abs(a02)		# 
		return Ak

	PM("## Amplitudenspektrum")

	P(f"f({L(t)}) = {L(a_k + b_k + a02)}")

	Ak = A_k()

	for v in V:
		P(f"A_{{{L(v)}}} = {L(Ak(v))}")


# - - - -

## Aufgabe 6 #AS-SUM

def A6_AS_SUM(S:sp.Expr, k:sp.Symbol, t:sp.Symbol, *V:int):
	def chop_shop(S:sp.Expr) -> tuple[sp.Expr, sp.Expr, sp.Expr]:
		def fetch_SumOf(func) -> sp.Expr:
			m = lambda t: isinstance(t, func)
			Match = lambda term: any(map(m, term.args[0].as_terms()[-1]))
			for term in S.as_terms()[-1]:
				if isinstance(term, sp.Sum) and Match(term):
					inner = term.args[0]
					return S.coeff(term) * inner
			return sp.Number(0)
		a_k = fetch_SumOf(sp.cos)
		b_k = fetch_SumOf(sp.sin)
		isSum = lambda S: any(map(lambda t: isinstance(t, sp.Sum), S.args)) # type: ignore
		a0_2 = sum(term for term in S.args if not isSum(term) and not isinstance(term, sp.Sum)) # type: ignore
		return a_k, b_k, a0_2

	def A_k():
		def fac(S:sp.Expr):
			if len(S.args) == 0:
				return 0
			f = lambda term: 1 if isinstance(term, sp.cos) or isinstance(term, sp.sin) else term
			return math.prod([f(term) for term in S.args])

		a_k, b_k, a0_2 = chop_shop(S)
		a = fac(a_k)
		b = fac(b_k)


		A = sp.sqrt(a**2 + b**2) # type: ignore
		P(
			f"A_k = \\sqrt{{a_k^2 + b_k^2}} = " +
			f" \\sqrt{{({L(a)})^{{2}} + ({L(b)})^{{2}}}} = " +
			f" \\sqrt{{{L(a**2)} + {L(b**2)}}} = {L(A)}" # type: ignore
		)
		def Ak(i:int):
			if i != 0:
				return A.limit(k, i)
			return 2*abs(a0_2)		# 
		return Ak

	PM("## Amplitudenspektrum")

	P(f"f({L(t)}) = {L(S)}")

	Ak = A_k()

	for v in V:
		P(f"A_{{{L(v)}}} = {L(Ak(v))}")

# Example

# k = sp.symbols('k', integer = True, positive = True)
# 
# a_k = 2*sp.Sum(sp.cos(pi*k*t)/k, (k, 1, sp.oo))
# 
# b_k = -2*sp.Sum(sp.sin(pi*k*t)/k, (k, 1, sp.oo))		# the labels a_k, b_k doesn't really matters
# 
# a0_2 = R(1, 2)	# a0/2
# 
# S = a_k + b_k + a0_2
# 
# A6_AS_SUM(S, k, t, 0, 1)

# - - - - - - -

# Globals

t = sp.symbols('t')

# Q-1

In [ ]:
f = 8 * cos(R(2, 5)*pi*t) + 6 * sin(R(2, 3)*pi*t)	# R -> alias sympy.Rational

A1(f, t)

# Q-2

In [ ]:
f = sp.Piecewise(
    (-2 * abs(t - 7)         , ( 4 < t) & (t <= 10)),
    (-1                      , (10 < t) & (t <= 16)),
    ((t - 19)**2/2 - R(9, 2) , (16 < t) & (t <= 22))
)

A2(f, t)

# Q-3

In [ ]:
f = sp.Piecewise(       # DEF FROM [-T/2, T/2]
    ( 0, (-2 < t) & (t <= -1)),
    ( 2, (-1 < t) & (t <=  0)),
    (-2, ( 0 < t) & (t <=  1)),
    ( 0, ( 1 < t) & (t <=  2))
    )

coefficient = 3

A3(f, t, coefficient)

# Q-4

In [ ]:
f = sp.Piecewise(
    (-t/4, (-4 < t) & (t <= 0)),
    ( 0  , ( 0 < t) & (t <= 4)),
     )

coefficient = 4

A4(f, t, coefficient)

# Q-5

In [ ]:
f = sp.Piecewise((3 * sp.exp(t), (0 < t) & (t <= sp.ln(2))))

A5(f, t)

# Q-6

In [ ]:
k = sp.symbols('k', integer = True, positive = True)

a_k = sp.Sum(2*sp.cos(pi*k*t)/k, (k, 1, sp.oo))

b_k = sp.Sum(-2*sp.sin(pi*k*t)/k, (k, 1, sp.oo))		# the labels a_k, b_k doesn't really matters

a02 = R(1, 2)

A6(a_k, b_k, a02, k, t, 0, 1)

# 2*sqrt(2) muss auf moodle so `sqrt(2)^3` geschrieben!

In [ ]:
# as a sum

a_k = 2*sp.Sum(sp.cos(pi*k*t)/k, (k, 1, sp.oo))

b_k = -2*sp.Sum(sp.sin(pi*k*t)/k, (k, 1, sp.oo))		# the labels a_k, b_k doesn't really matters

a02 = R(1, 2)

S = a_k + b_k + a02	# Deﬁnition 16.4 (Grundschwingung und Oberschwingungen) S.581

A6_AS_SUM(S, k, t, 0, 1)